In [1]:
import os
import json
import psycopg2
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import folium
import fiona
from pyproj import Proj, transform
import osmnx as ox
import networkx as nx
import matplotlib.colors as colors
import matplotlib.cm as cm
from shapely.ops import cascaded_union
import geojson
import operator

In [2]:
nx.__version__

'2.1'

In [3]:
import matplotlib.cm as cmx
import matplotlib.colors as colors

def style_function(feature):
    color = feature['properties']['color']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': "rgb(0, 0, 255)" if color == 2 else "rgb(255, 0, 0)" 
    }

In [4]:
ox.config(log_file=True, log_console=False, use_cache=True)

In [5]:
location = [47.3754, 8.5413]#[47.22662, 8.81834]

In [6]:
tags = {
  "landuse_tags": [
    "retail"
  ],
  "amenity_tags": [
    "pub",
    "bar",
    "cafe",
    "restaurant",
    "pharmacy",
    "bank",
    "fast_food",
    "food_court",
    "ice_cream",
    "library",
    "ferry_terminal",
    "clinic",
    "doctors",
    "hospital",
    "pharmacy",
    "veterinary",
    "dentist",
    "arts_centre",
    "cinema",
    "community_centre",
    "casino",
    "fountain",
    "nightclub",
    "studio",
    "theatre",
    "dojo",
    "internet_cafe",
    "marketplace",
    "post_opffice",
    "townhall"
  ],
  "shop_tags": [
    "mall",
    "bakery",
    "beverages",
    "butcher",
    "chocolate",
    "coffee",
    "confectionery",
    "deli",
    "frozen_food",
    "greengrocer",
    "healthfood",
    "ice_cream",
    "pasta",
    "pastry",
    "seafood",
    "spices",
    "tea",
    "department_store",
    "supermarket",
    "bag",
    "boutique",
    "clothes",
    "fashion",
    "jewelry",
    "leather",
    "shoes",
    "tailor",
    "watches",
    "chemist",
    "cosmetics",
    "hairdresser",
    "medical_supply",
    "electrical",
    "hardware",
    "electronics",
    "sports",
    "swimming_pool",
    "collector",
    "games",
    "music",
    "books",
    "gift",
    "stationery",
    "ticket",
    "laundry",
    "pet",
    "tobacco",
    "toys"
  ],
  "leisure_tags": [
    "adult_gaming_centre",
    "amusement_arcade",
    "beach_resort",
    "fitness_centre",
    "garden",
    "ice_rink",
    "sports_centre",
    "water_park"
  ]
}


In [45]:
class AoiQueries():
    def __init__(self, location=None, tags=[], dbscan_eps=50, dbscan_minpoints=3):
        self.location = location
        self.tags = tags
        self.dbscan_eps = dbscan_eps
        self.dbscan_minpoints = dbscan_minpoints


    def _polygons_query(self):
        return """
(SELECT way AS geometry FROM planet_osm_polygon
    WHERE (amenity = ANY(ARRAY{amenity_tags})
            OR shop = ANY(ARRAY{shop_tags})
            OR leisure = ANY(ARRAY{leisure_tags})
            OR landuse = ANY(ARRAY{landuse_tags}))

           AND access IS DISTINCT FROM 'private'
           AND st_within(way, {bbox}))

UNION ALL

(SELECT polygon.way AS geometry FROM planet_osm_polygon AS polygon
    INNER JOIN planet_osm_point AS point
        ON st_within(point.way, polygon.way)
    WHERE (point.amenity = ANY(ARRAY{amenity_tags})
            OR point.shop = ANY(ARRAY{shop_tags})
            OR point.leisure = ANY(ARRAY{leisure_tags})
            OR point.landuse = ANY(ARRAY{landuse_tags}))

        AND point.access IS DISTINCT FROM 'private'
        AND st_within(point.way, {bbox})
        AND polygon.building IS NOT NULL)
        """.format(bbox=self._bbox_query(), **self.tags)

    def _clusters_query(self):
        return """
WITH polygons AS ({polygons_query})
SELECT polygon.geometry AS geometry,
       ST_ClusterDBSCAN(polygon.geometry, eps := {eps}, minpoints := {minpoints}) over () AS cid
FROM polygons AS polygon
        """.format(polygons_query=self._polygons_query(), eps=self.dbscan_eps, minpoints=self.dbscan_minpoints)

    def _hulls_query(self):
        return """
WITH clusters AS ({clusters_query})
SELECT cid, ST_ConvexHull(ST_Union(geometry)) AS geometry
FROM clusters
WHERE cid IS NOT NULL
GROUP BY cid
        """.format(clusters_query=self._clusters_query())
    
    def _hulls_without_water_query(self):
        return """
WITH hulls AS ({hulls_query})
SELECT ST_Difference(hulls.geometry, coalesce((
    SELECT ST_Union(way) AS geometry FROM planet_osm_polygon
    WHERE (water IS NOT NULL OR waterway IS NOT NULL)
          AND (tunnel IS NULL OR tunnel = 'no')
    AND st_intersects(way, hulls.geometry)
), 'GEOMETRYCOLLECTION EMPTY'::geometry)) AS geometry
FROM hulls
        """.format(hulls_query=self._hulls_query())

    def _clusters_and_hulls_query(self):
        return """
WITH clusters AS ({clusters_query}),
hulls AS ({hulls_query})
SELECT cid, geometry FROM clusters
UNION ALL
SELECT cid, geometry FROM hulls
        """.format(clusters_query=self._clusters_query(), hulls_query=self._hulls_query())
    
    def _bbox_query(self):
        location_3857 = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), self.location[1], self.location[0])
        location_3857 = " ".join([str(coordinate) for coordinate in location_3857])

        return """
    (SELECT ST_Buffer(ST_GeomFromText('POINT({})', 3857), 1000) AS bbox)
        """.format(location_3857)



In [46]:
hulls_query = AoiQueries(location=location, tags=tags, dbscan_eps=50)._hulls_query()
hulls_without_water_query = AoiQueries(location=location, tags=tags, dbscan_eps=50)._hulls_without_water_query()

In [47]:
print(hulls_query)


WITH clusters AS (
WITH polygons AS (
(SELECT way AS geometry FROM planet_osm_polygon
    WHERE (amenity = ANY(ARRAY['pub', 'bar', 'cafe', 'restaurant', 'pharmacy', 'bank', 'fast_food', 'food_court', 'ice_cream', 'library', 'ferry_terminal', 'clinic', 'doctors', 'hospital', 'pharmacy', 'veterinary', 'dentist', 'arts_centre', 'cinema', 'community_centre', 'casino', 'fountain', 'nightclub', 'studio', 'theatre', 'dojo', 'internet_cafe', 'marketplace', 'post_opffice', 'townhall'])
            OR shop = ANY(ARRAY['mall', 'bakery', 'beverages', 'butcher', 'chocolate', 'coffee', 'confectionery', 'deli', 'frozen_food', 'greengrocer', 'healthfood', 'ice_cream', 'pasta', 'pastry', 'seafood', 'spices', 'tea', 'department_store', 'supermarket', 'bag', 'boutique', 'clothes', 'fashion', 'jewelry', 'leather', 'shoes', 'tailor', 'watches', 'chemist', 'cosmetics', 'hairdresser', 'medical_supply', 'electrical', 'hardware', 'electronics', 'sports', 'swimming_pool', 'collector', 'games', 'music', 'books'

In [38]:
with psycopg2.connect("") as conn:
    aois = gpd.read_postgis(hulls_query, conn, geom_col='geometry')
    
    aois.crs = fiona.crs.from_epsg(3857)
    
    aois_4326 = aois.to_crs(fiona.crs.from_epsg(4326))


In [39]:
m = folium.Map(location=location, zoom_start=16)

folium.GeoJson(aois).add_to(m)

m

In [44]:
def bbox_query():
        location_3857 = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), location[1], location[0])
        location_3857 = " ".join([str(coordinate) for coordinate in location_3857])

        return """
    (SELECT ST_Buffer(ST_GeomFromText('POINT({})', 3857), 1000) AS bbox)
        """.format(location_3857)
    

water_query = """
SELECT ST_Union(way) AS geometry FROM planet_osm_polygon
WHERE (water IS NOT NULL OR waterway IS NOT NULL)
      AND (tunnel IS NULL OR tunnel = 'no')
AND st_intersects(way, {bbox})
""".format(bbox=bbox_query())

with psycopg2.connect("") as conn:
    aois = gpd.read_postgis(water_query, conn, geom_col='geometry')
    
    aois.crs = fiona.crs.from_epsg(3857)
    
    aois_4326 = aois.to_crs(fiona.crs.from_epsg(4326))
    
m = folium.Map(location=location, zoom_start=16)

folium.GeoJson(aois).add_to(m)

m

In [48]:
with psycopg2.connect("") as conn:
    aois = gpd.read_postgis(hulls_without_water_query, conn, geom_col='geometry')
    
    aois.crs = fiona.crs.from_epsg(3857)
    
    aois_4326 = aois.to_crs(fiona.crs.from_epsg(4326))


In [49]:
m = folium.Map(location=location, zoom_start=16)

folium.GeoJson(aois).add_to(m)

m